<a href="https://colab.research.google.com/github/ndavila/Neutron_Capture_Cross_Sections/blob/main/neutron_capture_cross_section.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a notebook that calaculates neutron capture cross sections